In [1]:
import java.nio.file.Paths
import ammonite.ops._
import ammonite._

val p = Paths.get("/home/ixaxaar/.m2/repository/ml/dmlc/mxnet/mxnet-full_2.11-linux-x86_64-gpu/0.1.2-SNAPSHOT/mxnet-full_2.11-linux-x86_64-gpu-0.1.2-SNAPSHOT.jar")

interp.load.cp(FilePath(p).asInstanceOf[ammonite.ops.Path])

import java.nio.file.Paths

import ammonite.ops._

import ammonite._


p: java.nio.file.Path = /home/ixaxaar/.m2/repository/ml/dmlc/mxnet/mxnet-full_2.11-linux-x86_64-gpu/0.1.2-SNAPSHOT/mxnet-full_2.11-linux-x86_64-gpu-0.1.2-SNAPSHOT.jar

In [12]:
import ml.dmlc.mxnet._
import ml.dmlc.mxnet.{ Symbol => T }
import ml.dmlc.mxnet.{ NDArray => np }

import ml.dmlc.mxnet._

import ml.dmlc.mxnet.{ Symbol => T }

import ml.dmlc.mxnet.{ NDArray => np }

# Data

In [67]:
np.ones(2, 3)
np.zeros(3, 3)
val x = np.array(List(1,2,3,4).toArray.map{ _.toFloat }, shape=Shape(2,2))
x.shape

res66_0: NDArray = ml.dmlc.mxnet.NDArray@b56d6c4f
res66_1: NDArray = ml.dmlc.mxnet.NDArray@7524520f
x: NDArray = ml.dmlc.mxnet.NDArray@e4218aff
res66_3: Shape = (2,2)

In [28]:
val x = np.ones(100, 100) + np.zeros(100, 100)
val y = np.ones(100, 100) * np.zeros(100, 100)
val z = np.ones(100, 100) / 3 + 1
x += (y + z)
np.dot(x, z).toArray

x: NDArray = ml.dmlc.mxnet.NDArray@907ad41
y: NDArray = ml.dmlc.mxnet.NDArray@713fff9
z: NDArray = ml.dmlc.mxnet.NDArray@e7bd815c
res27_3: NDArray = ml.dmlc.mxnet.NDArray@10b08418
res27_4: Array[Float] = Array(
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
...

In [54]:
val x = np.ones(10, 5)
x.shape
x.toArray.length
x.slice(0).shape
x.slice(1).toArray.length
x.slice(0, 2).toArray.length
x.reshape(Shape(5, 10)).shape

x: NDArray = ml.dmlc.mxnet.NDArray@fd2cfa66
res53_1: Shape = (10,5)
res53_2: Int = 50
res53_3: Shape = (1,5)
res53_4: Int = 5
res53_5: Int = 10
res53_6: Shape = (5,10)

In [58]:
np.save("/home/ixaxaar/src/notebooks/ndarray.mxnet", List(x))
val y = np.load("/home/ixaxaar/src/notebooks/ndarray.mxnet")._2.head.shape

y: Shape = (10,5)

In [66]:
x.context
val ctx = Context.gpu(0)
np.array(x.toArray, x.shape, ctx).context

val gpu_x = np.zeros(x.shape, ctx)
x.copyTo(gpu_x)
gpu_x.context

res65_0: Context = cpu(0)
ctx: Context = gpu(0)
res65_2: Context = gpu(0)
gpu_x: NDArray = ml.dmlc.mxnet.NDArray@2a9c2d46
res65_4: NDArray = ml.dmlc.mxnet.NDArray@2ea112fa
res65_5: Context = gpu(0)

# Model API

In [10]:
val data = T.Variable("data")
val f1 = T.FullyConnected(name="f1")()(Map("data" -> data, "num_hidden" -> 128))
val a1 = T.Activation(name="relu1")()(Map("data" -> f1, "act_type" -> "relu"))
val f2 = T.FullyConnected(name="f2")()(Map("data" -> a1, "num_hidden" -> 128))
val softmax = T.SoftmaxOutput(name="softmax")()(Map("data" -> f2))

data: Symbol = ml.dmlc.mxnet.Symbol@687797cb
f1: Symbol = ml.dmlc.mxnet.Symbol@2841c40d
a1: Symbol = ml.dmlc.mxnet.Symbol@c690d46
f2: Symbol = ml.dmlc.mxnet.Symbol@76dcd821
softmax: Symbol = ml.dmlc.mxnet.Symbol@2241c0fe

In [10]:
val model = FeedForward.newBuilder(softmax)
    .setContext(Context.gpu())
    .setNumEpoch(10)
    .setOptimizer(new Adam())
    .setTrainData(trainDataIter)
    .setEvalData(evalDataIter)
    .build()

cmd10.sc:4: not found: type Adam
    .setOptimizer(new Adam())
                      ^cmd10.sc:5: not found: value trainDataIter
    .setTrainData(trainDataIter)
                  ^cmd10.sc:6: not found: value evalDataIter
    .setEvalData(evalDataIter)
                 ^

: 

In [10]:
model.fit(trainData = train)

cmd10.sc:1: not found: value model
val res10 = model.fit(trainData = train)
            ^cmd10.sc:1: not found: value trainData
val res10 = model.fit(trainData = train)
                      ^

: 

In [10]:
val modelPrefix: String = "checkpt"
val num_epoch = 100
Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)

// load model back
val model_loaded = FeedForward.load(modelPrefix, num_epoch)

cmd10.sc:3: not found: value epoch
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                ^cmd10.sc:3: not found: value symbol
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                           ^cmd10.sc:3: not found: value argParams
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                                   ^cmd10.sc:3: not found: value auxStates
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                                              ^

: 